In [ ]:
# Импорт библиотек
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tqdm import tqdm

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Настройка отображения
pd.set_option('display.max_columns', None)
sns.set_style('whitegrid')

# Проверка доступности GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

In [ ]:
# Загрузка данных
train_df = pd.read_csv('/kaggle/input/playground-series-s5e10/train.csv')

print(f"Размер датасета: {train_df.shape}")
print(f"\nПервые строки:")
print(train_df.head())
print(f"\nИнформация о данных:")
print(train_df.info())
print(f"\nПропущенные значения:")
print(train_df.isnull().sum())


In [ ]:
# Подготовка данных для нейросети

# Разделение на признаки и целевую переменную
X = train_df.drop(['id', 'accident_risk'], axis=1)
y = train_df['accident_risk'].values

print("Типы признаков:")
print(X.dtypes)

# Определяем категориальные и числовые признаки
categorical_features = X.select_dtypes(include=['object', 'bool']).columns.tolist()
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nКатегориальные признаки: {categorical_features}")
print(f"Числовые признаки: {numerical_features}")

# Кодирование категориальных признаков с помощью Label Encoding
X_encoded = X.copy()
label_encoders = {}

for col in categorical_features:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    label_encoders[col] = le

print(f"\nРазмер после кодирования: {X_encoded.shape}")
print(f"Количество признаков: {X_encoded.shape[1]}")

In [ ]:
# Нормализация числовых признаков
scaler = StandardScaler()
X_scaled = X_encoded.copy()
X_scaled[numerical_features] = scaler.fit_transform(X_encoded[numerical_features])

print("Данные после нормализации:")
print(X_scaled.head())
print(f"\nСтатистика числовых признаков после нормализации:")
print(X_scaled[numerical_features].describe())

In [ ]:
# Разделение на train и validation
X_train, X_val, y_train, y_val = train_test_split(
    X_scaled, y, 
    test_size=0.2, 
    random_state=42
)

print(f"Train размер: {X_train.shape}")
print(f"Validation размер: {X_val.shape}")

# Конвертация в numpy arrays
X_train_np = X_train.values.astype(np.float32)
X_val_np = X_val.values.astype(np.float32)
y_train_np = y_train.astype(np.float32).reshape(-1, 1)
y_val_np = y_val.astype(np.float32).reshape(-1, 1)


In [ ]:
# Определение архитектуры нейросети
class AccidentRiskPredictor(nn.Module):
    def __init__(self, input_dim):
        super(AccidentRiskPredictor, self).__init__()
        
        self.network = nn.Sequential(
            # Input layer 
            nn.Linear(input_dim, 512), 
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.1),  
            
            nn.Linear(512, 256), 
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(256, 128), 
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.1),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(0.05),
            
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            # Без dropout на последних слоях
            
            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(),
            
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return self.network(x)

# Создание модели
input_dim = X_train_np.shape[1]
model = AccidentRiskPredictor(input_dim).to(device)

print(f"\nКоличество параметров: {sum(p.numel() for p in model.parameters())}")


In [ ]:
# Настройка обучения
criterion = nn.MSELoss()  
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=15, factor=0.5, verbose=True)

# Параметры обучения
batch_size = 512
num_epochs = 200
best_val_loss = float('inf')
patience = 50
patience_counter = 0

# Создание DataLoader для батчевого обучения
train_dataset = TensorDataset(
    torch.FloatTensor(X_train_np), 
    torch.FloatTensor(y_train_np)
)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Для валидации
X_val_tensor = torch.FloatTensor(X_val_np).to(device)
y_val_tensor = torch.FloatTensor(y_val_np).to(device)

print(f"Параметры обучения:")
print(f"Batch size: {batch_size}")
print(f"Количество эпох: {num_epochs}")
print(f"Learning rate: 0.001")
print(f"Оптимизатор: Adam")
print(f"Функция потерь: MSE")
print(f"Device: {device}")


In [ ]:
# Обучение модели
train_losses = []
val_losses = []

print("Начало обучения...\n")

for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    
    for batch_X, batch_y in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        # Forward pass
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item() * batch_X.size(0)
    
    train_loss = train_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = criterion(val_outputs, y_val_tensor)
        val_losses.append(val_loss.item())
    
    # Learning rate scheduling
    scheduler.step(val_loss)
    
    # Вывод прогресса
    if (epoch + 1) % 10 == 0:
        train_rmse = np.sqrt(train_loss)
        val_rmse = np.sqrt(val_loss.item())
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'  Train Loss: {train_loss:.6f} (RMSE: {train_rmse:.6f})')
        print(f'  Val Loss: {val_loss.item():.6f} (RMSE: {val_rmse:.6f})')
        print(f'  LR: {optimizer.param_groups[0]["lr"]:.6f}\n')
    
    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Сохранение лучшей модели
        best_model_state = model.state_dict().copy()
    else:
        patience_counter += 1
        
    if patience_counter >= patience:
        print(f'Early stopping на эпохе {epoch+1}')
        # Восстановление лучшей модели
        model.load_state_dict(best_model_state)
        break

print(f"\nОбучение завершено!")
print(f"Лучший Validation Loss: {best_val_loss:.6f} (RMSE: {np.sqrt(best_val_loss.item()):.6f})")


In [ ]:
# Визуализация процесса обучения
plt.figure(figsize=(14, 5))

# График функции потерь
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss', linewidth=2)
plt.plot(val_losses, label='Validation Loss', linewidth=2)
plt.xlabel('Эпоха')
plt.ylabel('MSE Loss')
plt.title('Функция потерь во время обучения')
plt.legend()
plt.grid(True, alpha=0.3)

# График RMSE
plt.subplot(1, 2, 2)
train_rmse = [np.sqrt(loss) for loss in train_losses]
val_rmse = [np.sqrt(loss) for loss in val_losses]
plt.plot(train_rmse, label='Train RMSE', linewidth=2)
plt.plot(val_rmse, label='Validation RMSE', linewidth=2)
plt.xlabel('Эпоха')
plt.ylabel('RMSE')
plt.title('RMSE во время обучения')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Оценка модели
model.eval()
with torch.no_grad():
    # Предсказания на train
    X_train_tensor = torch.FloatTensor(X_train_np).to(device)
    y_train_pred = model(X_train_tensor).cpu().numpy()
    
    # Предсказания на validation
    y_val_pred = model(X_val_tensor).cpu().numpy()

# Вычисление метрик
print("МЕТРИКИ НА TRAIN:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_train_np, y_train_pred)):.6f}")
print(f"MAE: {mean_absolute_error(y_train_np, y_train_pred):.6f}")
print(f"R2 Score: {r2_score(y_train_np, y_train_pred):.6f}")

print("МЕТРИКИ НА VALIDATION:")
print(f"RMSE: {np.sqrt(mean_squared_error(y_val_np, y_val_pred)):.6f}")
print(f"MAE: {mean_absolute_error(y_val_np, y_val_pred):.6f}")
print(f"R2 Score: {r2_score(y_val_np, y_val_pred):.6f}")


In [ ]:
# Визуализация предсказаний
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Train
axes[0].scatter(y_train_np, y_train_pred, alpha=0.3, s=5)
axes[0].plot([y_train_np.min(), y_train_np.max()], 
             [y_train_np.min(), y_train_np.max()], 'r--', lw=2)
axes[0].set_xlabel('Фактические значения')
axes[0].set_ylabel('Предсказанные значения')
axes[0].set_title(f'Train: Предсказания vs Факт (R2={r2_score(y_train_np, y_train_pred):.4f})')
axes[0].grid(True, alpha=0.3)

# Validation
axes[1].scatter(y_val_np, y_val_pred, alpha=0.3, s=5)
axes[1].plot([y_val_np.min(), y_val_np.max()], 
             [y_val_np.min(), y_val_np.max()], 'r--', lw=2)
axes[1].set_xlabel('Фактические значения')
axes[1].set_ylabel('Предсказанные значения')
axes[1].set_title(f'Validation: Предсказания vs Факт (R2={r2_score(y_val_np, y_val_pred):.4f})')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Анализ ошибок
errors = y_val_np.flatten() - y_val_pred.flatten()

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Распределение ошибок
axes[0].hist(errors, bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(x=0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('Ошибка предсказания (Факт - Прогноз)')
axes[0].set_ylabel('Частота')
axes[0].set_title(f'Распределение ошибок\nСреднее: {errors.mean():.6f}, Std: {errors.std():.6f}')
axes[0].grid(True, alpha=0.3)

# Ошибки vs предсказанные значения
axes[1].scatter(y_val_pred, errors, alpha=0.3, s=5)
axes[1].axhline(y=0, color='r', linestyle='--', linewidth=2)
axes[1].set_xlabel('Предсказанные значения')
axes[1].set_ylabel('Ошибка (Факт - Прогноз)')
axes[1].set_title('Остатки vs Предсказания')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Загрузка тестовых данных и предсказание
try:
    test_df = pd.read_csv('data/test.csv')
    print(f"Тестовый набор загружен: {test_df.shape}")
      
    # Сохраняем ID
    test_ids = test_df['id']
    
    # Подготовка тестовых данных
    X_test = test_df.drop(['id', 'accident_risk'], axis=1, errors='ignore')
    
    # Кодирование категориальных признаков
    X_test_encoded = X_test.copy()
    for col in categorical_features:
        if col in X_test_encoded.columns:
            X_test_encoded[col] = label_encoders[col].transform(X_test_encoded[col].astype(str))
    
    # Нормализация числовых признаков
    X_test_scaled = X_test_encoded.copy()
    X_test_scaled[numerical_features] = scaler.transform(X_test_encoded[numerical_features])
    
    # Конвертация в numpy и тензоры
    X_test_np = X_test_scaled.values.astype(np.float32)
    X_test_tensor = torch.FloatTensor(X_test_np).to(device)
    
    # Предсказания
    model.eval()
    with torch.no_grad():
        test_predictions = model(X_test_tensor).cpu().numpy().flatten()
    
    # Создание submission файла
    submission = pd.DataFrame({
        'id': test_ids,
        'accident_risk': test_predictions
    })
    
    submission.to_csv('submission.csv', index=False)
    print("\nПредсказания сохранены в submission.csv")
    
    print(f"\nПервые строки submission:")
    print(submission.head(10))
    print(f"\nСтатистика предсказаний:")
    print(submission['accident_risk'].describe())
    print(f"\nРазмер submission: {submission.shape}")
    
except FileNotFoundError:
    print("Файл test.csv не найден. Пропускаем создание submission.")
